In [1]:
import codecs
#處理編碼的套件
import operator
##處理字典檔排序的套件
 

text_new =""

def cutSentence(string, keywords): ##放入原始文章路徑, 增加斷詞的list
#     text = codecs.open(text_path,"r","utf-8")   #開檔 #讀取存成TXT檔的文字，讀入後統一轉成UTF-8格式
    sentence = ""
    textList = []
       
#     for line in text.readlines():
#         line = line.strip() ##清除空白

    line = string
        
    for keyword in keywords:  #清除關鍵字
        line = "".join(line.split(keyword))

    for word in line:
        if word not in cutlist: #如果文字不是標點符號，就把字加到句子中
            sentence += word
            #print sentence
        else:
            textList.append(sentence) #如果遇到標點符號，把句子加到 text list中
            sentence = ""
            #print textList
    return textList#傳回一個文字陣列

In [2]:
def ngram(textLists,n,minFreq): #第一個參數放處理好的文章(LIST檔，utf-8編碼)，第二個參數放字詞的長度單位，第三個參數放至少要幾次以上
 
    words=[]     #存放擷取出來的字詞
    words_freq={}#存放字詞:計算個數 
    result= []
    for textList in textLists:
        for w in range(len(textList)-(n-1)): #要讀取的長度隨字詞長度改變
            words.append(textList[w:w+n]) #抓取長度w-(n-1)的字串

    for word in words:
        if word not in words_freq:               #如果這個字詞還沒有被放在字典檔中
            words_freq[word] = words.count(word) #就開一個新的字詞，裡面放入字詞計算的頻次
 
    words_freq = sorted(words_freq.items(),key=operator.itemgetter(1),reverse=True) #change words_freq from dict to list 
    
    for word in words_freq:
        if word[1] >= minFreq:
            result.append(word)
            
    return result ##回傳一個陣列[詞,頻次]

In [3]:
def longTermPriority(string, maxTermLength, minFreq):
    longTerms=[]          #長詞
    longTermsFreq=[]      #長詞+次數分配
    
    for i in range(maxTermLength,1,-1):
        text_list = cutSentence(string,longTerms)
#         print len(text_list)
        words_freq = ngram(text_list,i, minFreq)
        #print i

    
        for word_freq in words_freq:
            longTerms.append(word_freq[0])
            #print word_freq[0]
            longTermsFreq.append(word_freq) 
            #print word_freq

        return longTermsFreq

In [5]:
from pymongo import MongoClient

IP = 'localhost'
client = MongoClient(IP, 27017)

# Select database
db = client.test

# Select collection
collection = db.test

datas = collection.find()
with open("E:\\PythonWin\\workspace\\dict\\uselessDict\\specialSignList.txt","r",encoding='utf-8') as fr:
    stopWd = fr.read().strip().split('\n')
    cutlist = [signal for signal in stopWd if len(stopWd) > 0]

total_word=""
for idx, data in enumerate(datas):
    if idx <100:
        data['content'] = data['content'].lower().replace('-', '_')
        total_word += data['content']
#         print(data['content'])
#         print(total_word)
longTermFreq = longTermPriority(total_word,5,10) 
print(longTermFreq)

# with open(u"E:\\PythonWin\\workspace\\ucartitle_test2.csv",'w',encoding='utf-8') as fw:
#     for k,v in longTermFreq:
#         fw.write("{},{}".format(k,v)+'\n')

[('     ', 184), ('.....', 25), ('civic', 12), ('行車記錄器', 11)]


In [ ]:
with open(u"E:\\PythonWin\\workspace\\1Wd.csv",'w') as fw:
    for k,v in longTermFreq:
        fw.write("{},{}".format(k,v)+'\n')

In [ ]:
with open("E:\\PythonWin\\workspace\\dict\\uselessDict\\specialSignList.txt","r",encoding='utf-8') as fr:
    stopWd = fr.read().strip().split('\n')
    cutlist = [signal for signal in stopWd if len(signal) > 0]
print(cutlist)